# 1. 프로젝트 목표
- https://www.kaggle.com/c/competitive-data-science-predict-future-sales#evaluation 참고

- item_cnt_month 를 예측하는 것


# 2.문제

- https://www.kaggle.com/c/competitive-data-science-predict-future-sales/data 참고

-데이터는 files 폴더에 넣어뒀습니다.
  
  item_categories.csv (아이템의 카테고리와 아이템카테고리ID)
  
  items.csv(아이템이름,아이템ID,아이템 카테고리 ID )
  
  shops.csv(가게이름, 가게ID)
  
  sales_train.csv.gz(트레인 셋 - 날짜, 년-월기준 식별코드,가게이름, 아이템이름, 아이템가격,일별 판매량 )
  
  sample_submisson.csv(제출샘플, 테스트셋에서 id와 item_cnt_month의 정답)
  
  test.csv.gz(테스트셋)
  
  이렇게 구성되어있습니다.
  
  
# 3.할일
  
  - 트레인셋,테스트셋의 피쳐를 변형시키는 작업을 한 뒤 linear regression 모델을 적용시킵니다.
  
  - 팀회의를 통해 가설을 바탕으로 피쳐를 늘이거나 줄입니다.
  
  - 가설을 세우고 결과를 도출해 낸 후, 다른 가설을 세워 같은 작업을 반복합니다. 
  

3.일정

- 3/24 역할분담 토의, 분담한 역할 마감일 정하기


# 가설
- 카테고리별로 월별 판매량이 다를것이다
- 가격에 따라 판매량에 차이가 있을것이다.
- 온라인, 오프라인에 따라 판매량의 차이가 있을것이다
- 지역별로 온라인이 더 잘되는곳, 오프라인이 더 잘되는곳 등이 있을것이다

# Import data

In [194]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from googletrans import Translator 

In [219]:
df_test = ("../files/test.csv")
df_train = ("../files/sales_train_v2.csv")

df_test = pd.read_csv(df_test, sep = ",") #(214200, 3)
df_train = pd.read_csv(df_train, sep = ",") # (2935849, 6) 

In [220]:
df_items = ("../files/items.csv")
df_item_categories = ("../files/item_categories.csv")
df_shops = ("../files/shops.csv")

df_items = pd.read_csv(df_items, sep = ",") #(214200, 3)
df_item_categories = pd.read_csv(df_item_categories, sep = ",") # (2935849, 6) 
df_shops = pd.read_csv(df_shops, sep = ",") #(214200, 3)

In [221]:
df_item_categories.head()

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [222]:
df_items.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


# Translate df_item_categories, df_shop_name

In [223]:
translator = Translator()
item_category_name_ko = pd.Series([str(translator.translate(category_name, dest = "ko").text) for category_name in df_item_categories.item_category_name])
df_item_categories.item_category_name = item_category_name_ko
df_item_categories.item_category_name


shop_name_ko = pd.Series([str(translator.translate(shop_nm, dest = "ko").text) for shop_nm in df_shops.shop_name])
df_shops.shop_name = shop_name_ko
df_shops.shop_name

0                        Yakutsk Ordzhonikidze, 56 프랑
1                                     야쿠 츠크 TC "중앙"프랑
2                                  아디 게아 쇼핑 센터 "Mega"
3                              발라 시카 TRK "10 월 키노 미르"
4            볼츠키 (Volzhsky) 쇼핑 센터 "볼가 몰 (Volga Mall)"
5                       볼 로그 다 쇼핑 및 엔터테인먼트 센터 "마멀레이드"
6                          보로 네즈 (Porkhanovskaya, 13)
7                                Voronezh TRC "막시 미르"
8                            Voronezh TRC 도시 공원 "졸업생"
9                                               외환 거래
10                          Zhukovsky Str. 차 칼로프 39m?
11                       Zhukovsky Str. Chkalov 39 m²
12                                       온라인 상점 비상 사태
13                                    카잔 TC "Bahetle"
14                                   카잔 TC "파크 하우스"II
15                                   칼루가 TRC "XXI 세기"
16                                          콜로세움 "리오"
17    크라스 노야 르 스크 (Krasnoyarsk) 쇼핑 센터 "Vzletka Plaza"
18             크라스 노야 르 스크 (

# item_id - merge df_items, df_item_categories into df_item_info

In [224]:
#item_category와 item을 merge
df_item_info = pd.merge(df_item_categories, df_items, on = "item_category_id")
df_item_info.head(5)

,item_category_name,item_category_id,item_name,item_id
0,PC - 헤드셋 / 헤드폰,0,PC: Гарнитура HyperX Cloud Core игровая стерео...,5441
1,PC - 헤드셋 / 헤드폰,0,Наушники PHILIPS SBC HC8680,16255
2,PC - 헤드셋 / 헤드폰,0,Наушники RITMIX RH-120,16256
3,PC - 헤드셋 / 헤드폰,0,Наушники RITMIX RH-124 Black,16257
4,액세서리 - PS2,1,PS2: Карта памяти 8 Мб черная (Memory Card 8Mb...,5606


In [225]:
df_train = pd.merge(df_train,df_item_info ,on = "item_id")
df_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_category_name,item_category_id,item_name
0,02.01.2013,0,59,22154,999.0,1.0,시네마 - 블루 레이,37,ЯВЛЕНИЕ 2012 (BD)
1,23.01.2013,0,24,22154,999.0,1.0,시네마 - 블루 레이,37,ЯВЛЕНИЕ 2012 (BD)
2,20.01.2013,0,27,22154,999.0,1.0,시네마 - 블루 레이,37,ЯВЛЕНИЕ 2012 (BD)
3,02.01.2013,0,25,22154,999.0,1.0,시네마 - 블루 레이,37,ЯВЛЕНИЕ 2012 (BD)
4,03.01.2013,0,25,22154,999.0,1.0,시네마 - 블루 레이,37,ЯВЛЕНИЕ 2012 (BD)


In [226]:
df_test = pd.merge(df_test,df_item_info,on = "item_id")
df_test.head()

,ID,shop_id,item_id,item_category_name,item_category_id,item_name
0,0,5,5037,게임 - PS3,19,"NHL 15 [PS3, русские субтитры]"
1,5100,4,5037,게임 - PS3,19,"NHL 15 [PS3, русские субтитры]"
2,10200,6,5037,게임 - PS3,19,"NHL 15 [PS3, русские субтитры]"
3,15300,3,5037,게임 - PS3,19,"NHL 15 [PS3, русские субтитры]"
4,20400,2,5037,게임 - PS3,19,"NHL 15 [PS3, русские субтитры]"


#  shop_id - merge df_shop to  df_train, df_test 

In [227]:
df_train = pd.merge(df_train, df_shops, on = "shop_id")
df_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_category_name,item_category_id,item_name,shop_name
0,02.01.2013,0,59,22154,999.0,1.0,시네마 - 블루 레이,37,ЯВЛЕНИЕ 2012 (BD),"야로 슬라 블 쇼핑 센터 ""Altair"""
1,16.05.2013,4,59,2573,249.0,1.0,음악 - 현지 제작 CD,55,DEL REY LANA Born To Die,"야로 슬라 블 쇼핑 센터 ""Altair"""
2,26.01.2013,0,59,2574,399.0,1.0,음악 - 현지 제작 CD,55,DEL REY LANA Born To Die The Paradise Editio...,"야로 슬라 블 쇼핑 센터 ""Altair"""
3,09.01.2013,0,59,2574,399.0,1.0,음악 - 현지 제작 CD,55,DEL REY LANA Born To Die The Paradise Editio...,"야로 슬라 블 쇼핑 센터 ""Altair"""
4,24.02.2013,1,59,2574,399.0,1.0,음악 - 현지 제작 CD,55,DEL REY LANA Born To Die The Paradise Editio...,"야로 슬라 블 쇼핑 센터 ""Altair"""


In [228]:
df_test = pd.merge(df_test, df_shops, on = "shop_id")
df_test.head()

,ID,shop_id,item_id,item_category_name,item_category_id,item_name,shop_name
0,0,5,5037,게임 - PS3,19,"NHL 15 [PS3, русские субтитры]","볼 로그 다 쇼핑 및 엔터테인먼트 센터 ""마멀레이드"""
1,1,5,5320,음악 - 현지 제작 CD,55,ONE DIRECTION Made In The A.M.,"볼 로그 다 쇼핑 및 엔터테인먼트 센터 ""마멀레이드"""
2,2,5,5233,게임 - PS3,19,"Need for Speed Rivals (Essentials) [PS3, русск...","볼 로그 다 쇼핑 및 엔터테인먼트 센터 ""마멀레이드"""
3,3,5,5232,게임 - XBOX 360,23,"Need for Speed Rivals (Classics) [Xbox 360, ру...","볼 로그 다 쇼핑 및 엔터테인먼트 센터 ""마멀레이드"""
4,4,5,5268,게임 - PS4,20,"Need for Speed [PS4, русская версия]","볼 로그 다 쇼핑 및 엔터테인먼트 센터 ""마멀레이드"""


## df_test와 df_train을 concat하지 않은 이유


- 아이템이름,가게이름 등을 merge시키면서 오름차순으로 배열된 아이템이름,가게이름등이 순서에 영향을 미친 것 같습니다.
 좀 더 깔끔하게 코드를 정리할 수 있으면 좋을 것 같아요! 


- 위 가설은 다른 조 분들과 대화하면서 얻은 아이디어들입니다. 예시를 바탕으로 좀 더 다양한 가설을 생각해봅시다 :)